In [31]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import jv
from scipy.signal import convolve
import sympy as s
from sympy import I, oo, re, im, Abs, arg
from sympy.simplify.fu import TR9 as contract_trig_sums, TR10i
s.init_printing() # set up the best possible printing method for sympy

# 0. Some reminders, common code and sources
For our $2\omega + 3\omega$ experiment, 

<span style="color:orangered"> $\forall l \in \mathbb{Z}, \quad \displaystyle \sum_{\substack{n, m \in \mathbb{Z}^{2}\\ 2n + 3m = l}} J_n\left(\alpha \right)J_m\left(\beta \right)e^{-i \left( n\varphi_1 + m \varphi_2\right)}$</span>

So here the idea is to set the junction @ 0 voltage, and then to look @ the emitted spectrum. 

The I-V characteristic is, as per the multiharmonic notebook : 

$I = I_0 \displaystyle \sum_{l\in \mathbb{Z}} \left| p_l \right| \sin\left(\theta_0 + \omega_J t + l\omega t + \mathrm{Arg}(p_l)\right)$.

Since $V = 0$, $\omega_J = 0$ and thus :

$I = I_0 \displaystyle \sum_{l\in \mathbb{Z}} \left| p_l \right| \sin\left(\theta_0 + l\omega t + \mathrm{Arg}(p_l)\right)$.

So looking at the components of the signal having frequency $\omega$ :

$I_\omega = I_0 \Big(\left|p_{-1}\right|\sin\big(\theta_0 - \omega t + \mathrm{Arg}\left(p_{-1}\right)\big) + \left|p_{1}\right|\sin\big(\theta_0 + \omega t + \mathrm{Arg}\left(p_{1}\right)\big) \Big)$

Note : since we are forcing $V_{DC} = 0$ and $I_{DC} = 0$, the DC term $I_{DC} = I_0 \left|p_0 \right| \sin\left(\theta_0 + \mathrm{Arg}\!\left(p_0\right) \right)$ means that <span style='color:royalblue'> $\theta_0 = -\mathrm{Arg}\!\left(p_0\right) + n\pi$ </span> so long as $\left| p_0 \right| \neq 0$


When disspiating through a resistance R, this gives a power of :

<span style="color:royalblue">$P_\omega = \frac{R I_0^2}{2}\Big( \left| p_{-1} \right| ^2 + \left| p_{1} \right| ^2 + 2\left|p_{-1} \right| \left|p_{1} \right| \cos \left( 2\theta_0 + \mathrm{Arg}\!\left( p_1\right) + \mathrm{Arg}\!\left( p_{-1}\right) \right) \Big)$</span>

### RESOURCES

For grouping terms together, check out Rewriting : https://docs.sympy.org/latest/modules/rewriting.html

For a whole bunch of trig subs, look into https://docs.sympy.org/latest/modules/simplify/fu.html , there's probably already a TR function that'll do the job.

In [3]:
alpha, beta, phi_1, phi_2, k, n, k_0, n_0, q, delta, omega, phi_JJ, t = s.symbols(r"\alpha \beta \varphi_{1} \varphi_{2} k n k_0 n_0 q \delta \omega \varphi_{JJ} t", real = True)

def sub_for_delta(expression):
    return expression.subs(phi_1, 2*delta/3 + 2*phi_2/3)

def sub_for_phis(expression):
    return expression.subs(delta, 3*phi_1/2 - phi_2)

def particular_k_n(l):
    return -l, l

def bessel_terms(l, max_index):
    # Lists bessel terms appearing in expansion for harmonic l for easier factoring
    k_0, n_0 = particular_k_n(l)
    return [s.besselj(3*q + k_0, alpha)*s.besselj(-2*q+n_0, beta) for q in range(max_index + 1)]

# 1. Complex formalism


In [27]:
p_general = s.Sum(s.besselj(3*q + k_0, alpha)*s.besselj(-2*q+n_0, beta)*s.exp(-I*((3*q + k_0)*phi_1 + (-2*q+n_0)*phi_2 + phi_JJ)),
         (q, -oo, oo))

def general_term_complex(k_0, n_0, max_index):
    if max_index is oo :
        return s.Sum(s.besselj(3*q + k_0, alpha)*s.besselj(-2*q+n_0, beta)*s.exp(-I*((3*q + k_0)*phi_1 + (-2*q+n_0)*phi_2 + phi_JJ)),
                     (q, -max_index, max_index))
    else : 
        return s.Sum(s.besselj(3*q + k_0, alpha)*s.besselj(-2*q+n_0, beta)*s.exp(-I*((3*q + k_0)*phi_1 + (-2*q+n_0)*phi_2 + phi_JJ)),
                     (q, -max_index, max_index)).doit()

def p_l(l, max_index):
    return general_term_complex(*particular_k_n(l), max_index)

def freq_component_complex(l, max_index):
    if l == 0 : 
        return p_l(l, max_index)
    else : 
        return p_l(l, max_index)*s.exp(l*I*omega*t) + p_l(-l, max_index)*s.exp(-l*I*omega*t)

## Order 0 Shapiro term

In [11]:
l = 0
order = 5

bessels = bessel_terms(l, order)

p_0 = freq_component_complex(l, order)

p_0 = sub_for_delta(p_0)
p_0 = s.collect(p_0, bessels)
contract_trig_sums(im(p_0))

## Order 1 Shapiro term
### $\alpha \rightarrow 0$

In [40]:
l = 1
order = 2

bessels = bessel_terms(l, order)

p_1 = freq_component_complex(l, order)
p_1 = s.collect(p_1, bessels)
p_1_real = im(p_1)
#p_1_real = sub_for_delta(p_1_real)
#p_1_real = contract_trig_sums(p_1_real)
p_1_limalpha = p_1_real.series(alpha, 0, 3)
p_1_limalpha = contract_trig_sums(TR10i(s.collect(p_1_limalpha, s.besselj(1, beta))))
p_1_limalpha
#print(s.latex(p_1_limalpha))

so :

<span style='color:orangered'>$V_{\omega} \underset{\alpha \rightarrow 0}{=} \alpha \sin{\left(\varphi_{JJ} \right)} \cos{\left(\omega t + \varphi_{1} - \varphi_{2} \right)} J_{1}\left(\beta\right) + \mathcal{O}\left(\alpha^{2}\right)$ </span>

### $\beta \rightarrow 0$ 

In [43]:
l = 1
order = 2

bessels = bessel_terms(l, order)

p_1 = freq_component_complex(l, order)
p_1 = s.collect(p_1, bessels)
p_1_real = im(p_1)
#p_1_real = sub_for_delta(p_1_real)
#p_1_real = contract_trig_sums(p_1_real)
contract_trig_sums(TR10i(p_1_real.series(beta, 0, 3)))

so :

<span style='color:orangered'>$V_{\omega} \underset{\beta \rightarrow 0}{=} \frac{\beta}{2}\left[\sin{\left(\varphi_{JJ} \right)} \cos{\left(\omega t + \varphi_{1} - \varphi_{2} \right)} J_{1}\left(\alpha\right) - \sin{\left(\omega t - 2 \varphi_{1} + \varphi_{2} \right)} \cos{\left(\varphi_{JJ} \right)} J_{2}\left(\alpha\right)\right]  \mathcal{O}(\beta^3)$ </span>

# 2. Sine formalism

In [19]:
p_general = s.Sum(s.besselj(3*q + k_0, alpha)*s.besselj(-2*q+n_0, beta)*s.sin(phi_JJ + (2*(3*q + k_0) + (-2*q+n_0)*3)*omega*t + (3*q + k_0 )*phi_1 + (-2*q+n_0)*phi_2),
         (q, -oo, oo))

def general_term_real(k_0, n_0, max_index):
    if max_index is oo :
        return s.Sum(s.besselj(3*q + k_0, alpha)*s.besselj(-2*q+n_0, beta)*s.sin(phi_JJ + (2*(3*q + k_0) + (-2*q+n_0)*3)*omega*t + (3*q + k_0 )*phi_1 + (-2*q+n_0)*phi_2),
                     (q, -max_index, max_index))
    else : 
        return s.Sum(s.besselj(3*q + k_0, alpha)*s.besselj(-2*q+n_0, beta)*s.sin(phi_JJ + (2*(3*q + k_0) + (-2*q+n_0)*3)*omega*t + (3*q + k_0 )*phi_1 + (-2*q+n_0)*phi_2),
                     (q, -max_index, max_index)).doit()


def freq_component_sine(l, max_index):
    if l == 0 :
        return general_term_real(*particular_k_n(l), max_index)
    else :
        return general_term_real(*particular_k_n(l), max_index) + general_term_real(*particular_k_n(-l), max_index)

In [7]:
l = 0
order = 3

bessels = bessel_terms(l, order)

I_DC = freq_component_sine(l, order) # generating the expression
I_DC = sub_for_delta(I_DC) # turning the 
I_DC = s.collect(I_DC, bessels)

# And here we're using Hongguang Fu's TR9 sub to turn sums of trigs to products of trigs
I_DC = contract_trig_sums(I_DC)
I_DC

In [25]:
l = 1
order = 2

bessels = bessel_terms(l, order)

I_DC = freq_component_sine(l, order) # generating the expression
#I_DC = sub_for_delta(I_DC) # turning the 
I_DC = s.collect(I_DC, bessels)

# And here we're using Hongguang Fu's TR9 sub to turn sums of trigs to products of trigs
I_DC = contract_trig_sums(I_DC)
I_DC.series(beta, 0, 3)